In [11]:
import glob, os
import pandas as pd
import boto3
from io import StringIO
from dateutil.parser import parse
from aws.s3 import create_s3_session
from aws.s3_task import list_exist_date_str, find_prev_date_str, load_and_clean_data
from aws.checker import duplicate_check
from aws.s3_writer import write_to_s3_from_file, write_clean_df
s3=create_s3_session()

## mass write


In [12]:
csvs_path = r"C:\Users\jy\GoogleDrive2\_mpdata\_mpdata"
csvs = glob.glob(os.path.join(csvs_path, "2*.csv"))

for j,current_csv in enumerate(csvs):
    current_df = pd.read_csv(current_csv, encoding="cp1256")
    current_date_str = os.path.basename(current_csv).split(".")[0].split("_")[0]
    current_clean_df = load_and_clean_data(current_date_str, current_df)

    prev_date_str = find_prev_date_str(current_date_str)
    if prev_date_str is not None:
        prev_clean_df = load_and_clean_data(prev_date_str)
        is_duplicated = duplicate_check(current_clean_df, prev_clean_df)

    else:
        is_duplicated = False
    
    is_exist_raw = current_date_str in list_exist_date_str(root_dir = "mplus")
    is_exist_clean = current_date_str in list_exist_date_str(root_dir = "clean_mplus")
    if is_exist_raw and is_exist_clean:
        print("Skip:", current_csv)
        
    if not is_duplicated:
        # write raw and clean df to s3
        write_to_s3_from_file(current_csv)
        write_clean_df(current_clean_df)
        print("Done:", current_csv)
    else:
        print("Duplicated:", current_date_str)
        
    
    

Skip: C:\Users\jy\GoogleDrive2\_mpdata\_mpdata\20151116.csv
Done: C:\Users\jy\GoogleDrive2\_mpdata\_mpdata\20151116.csv
Skip: C:\Users\jy\GoogleDrive2\_mpdata\_mpdata\20151117.csv


KeyboardInterrupt: 

In [8]:
keys = [
        obj["Key"]
        for obj in s3.list_objects(Bucket="malaysia-stock-eod-data")["Contents"]
        if obj["Key"].split("/")[0] == "mplus"
    ]
keys2 = [
        obj["Key"]
        for obj in s3.list_objects(Bucket="malaysia-stock-eod-data")["Contents"]
        if obj["Key"].split("/")[0] == "clean_mplus"
    ]
keys

['mplus/2015/11/16/data.csv',
 'mplus/2015/11/17/data.csv',
 'mplus/2015/11/18/data.csv',
 'mplus/2015/11/19/data.csv',
 'mplus/2021/05/05/data.csv',
 'mplus/2021/05/06/data.csv']

In [9]:
keys2

['clean_mplus/2015/11/16/data.csv',
 'clean_mplus/2015/11/17/data.csv',
 'clean_mplus/2015/11/18/data.csv',
 'clean_mplus/2015/11/19/data.csv',
 'clean_mplus/2021/05/06/data.csv']

In [2]:
import os

# Get environment variables
# AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
# AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [3]:
# session = boto3.Session(aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
# s3 = session.client("s3", region_name = "ap-southeast-1")
s3 = create_s3_session()
# response = s3.upload_file(
#     Filename = "C:/Users/jy/20210505_170708564037.csv",
#     Bucket="malaysia-stock-eod-data",
#     Key="mplus/2021/05/05/data.csv"
# )

In [4]:
keys = [obj["Key"] for obj in s3.list_objects(Bucket="malaysia-stock-eod-data")["Contents"]]

In [5]:
keys

['mplus/2021/05/05/data.csv', 'mplus/2021/05/06/data.csv']

In [75]:
response = s3.download_file(Filename = "./tempdata.csv", Bucket="malaysia-stock-eod-data", Key="mplus/2021/05/05/data.csv")

In [96]:
def parse_keypath(key):
    return "".join(key.split("/")[1:4])
parse_keypath(keys[0])

'20210505'

In [81]:
from dateutil.parser import parse


def load_from_s3(date_str):
    date_str = "2020-01-05"
    date_dt =  parse(date_str)
    date_param = date_dt.strftime("%Y/%m/%d")
    response = s3.get_object(Bucket="malaysia-stock-eod-data", Key=f"mplus/{date_param}/data.csv")
    csv_txt=response["Body"].read().decode('cp1256')
    df = pd.read_csv(StringIO(csv_txt), encoding="cp1256")
    return df


In [43]:
csv_txt=response["Body"].read().decode('cp1256')
csv_file = StringIO(csv_txt)
# df = pd.read_csv(csv_file, encoding="cp1256")

In [47]:
date = "20210505"
df = LoadMP(csv_file, date).fdf

In [64]:
cols = df.columns

In [49]:
from trade.load.mongo import initiate_mongo

In [51]:
client,coll = initiate_mongo()


In [60]:
cols2 = list(coll.find_one().keys())

In [61]:
len(cols2)

111

In [65]:
[j for j in cols if j not in cols2]


['RSSVol']